<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/02_sam2_visor_mascaras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =============================================================================
# SAM 2.0 VISUALIZADOR AVANZADO DE MÁSCARAS - VERSIÓN CORREGIDA
# =============================================================================
# Proyecto: Visualización y Análisis de Resultados SAM 2.0
# Funcionalidad: Carga NPZ + JSON, genera visualizaciones avanzadas
# Compatible con: Resultados del 02_SAM2_Evaluador_Master.ipynb
# =============================================================================

# INSTALACIÓN DE DEPENDENCIAS
# =============================================================================

!pip install -q opencv-python matplotlib Pillow
!pip install -q numpy scipy
!pip install -q seaborn plotly  # Para gráficas avanzadas
!pip install -q tqdm  # Barras de progreso

print("✅ Dependencias instaladas correctamente")

✅ Dependencias instaladas correctamente


In [1]:
# IMPORTACIONES Y MONTAJE DE DRIVE
# =============================================================================

import os
import sys
import json
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
import warnings

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import seaborn as sns
from tqdm.auto import tqdm

from google.colab import drive

# Configuración
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['font.size'] = 10

# Montar Drive
drive_path = Path("/content/drive/MyDrive")
if not drive_path.exists():
    print("📁 Montando Google Drive...")
    drive.mount('/content/drive')
    print("✅ Drive montado correctamente")
else:
    print("✅ Google Drive ya está montado")

print("✅ Configuración completada")

📁 Montando Google Drive...
Mounted at /content/drive
✅ Drive montado correctamente
✅ Configuración completada


In [2]:
# CELL 3: CONFIGURACIÓN DEL VISUALIZADOR
# =============================================================================

@dataclass
class ConfigVisualizador:
    """Configuración centralizada del visualizador"""

    # Rutas base
    BASE_PATH: Path = Path("/content/drive/MyDrive/TFM/sam2")
    RESULTADOS_PATH: Path = BASE_PATH / "resultados"
    IMAGENES_PATH: Path = BASE_PATH / "imagenes"
    OUTPUT_PATH: Path = BASE_PATH / "visualizaciones_avanzadas"

    # Opciones de visualización
    GENERAR_MASCARAS_INDIVIDUALES: bool = True
    GENERAR_OVERLAYS: bool = True
    GENERAR_RECORTES: bool = True
    GENERAR_ALPHA: bool = True
    GENERAR_COMPARATIVAS: bool = True
    GENERAR_ESTADISTICAS: bool = True

    # Colores para visualización (RGB)
    COLORES_MODELOS = {
        'tiny': (255, 0, 0),      # Rojo
        'small': (0, 255, 0),     # Verde
        'base_plus': (0, 0, 255), # Azul
        'large': (255, 0, 255)    # Magenta
    }

    # Transparencia para overlays
    ALPHA_OVERLAY: float = 0.5

    def __post_init__(self):
        """Crea directorios necesarios"""
        self.OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
        (self.OUTPUT_PATH / "por_imagen").mkdir(exist_ok=True)
        (self.OUTPUT_PATH / "comparativas").mkdir(exist_ok=True)
        (self.OUTPUT_PATH / "estadisticas").mkdir(exist_ok=True)

# Crear instancia global
config = ConfigVisualizador()
print(f"✅ Configuración creada")
print(f"📁 Salida: {config.OUTPUT_PATH}")

✅ Configuración creada
📁 Salida: /content/drive/MyDrive/TFM/sam2/visualizaciones_avanzadas


In [3]:
# CARGADOR DE RESULTADOS
# =============================================================================

class CargadorResultados:
    """Carga y organiza resultados de evaluaciones SAM 2.0"""

    def __init__(self, directorio_ejecucion: Path):
        self.directorio_ejecucion = Path(directorio_ejecucion)
        self.datos_cargados = {}

    def cargar_ejecucion_completa(self) -> Dict[str, Any]:
        print(f"\n{'='*80}")
        print(f"CARGANDO RESULTADOS DE: {self.directorio_ejecucion.name}")
        print(f"{'='*80}\n")

        modelos = ['tiny', 'small', 'base_plus', 'large']
        resultados = {}

        for modelo in modelos:
            dir_modelo = self.directorio_ejecucion / modelo

            if not dir_modelo.exists():
                print(f"⚠️  Modelo {modelo}: No encontrado (saltando)")
                continue

            print(f"📂 Cargando modelo: {modelo}")

            # Cargar JSON
            json_files = list((dir_modelo / "json").glob("*.json"))
            if not json_files:
                print(f"   ❌ No se encontraron archivos JSON")
                continue

            with open(json_files[0], 'r') as f:
                json_data = json.load(f)

            # Organizar por imagen
            imagenes_data = {}

            for resultado in json_data.get('resultados', []):
                if not resultado.get('exito'):
                    continue

                nombre_imagen = resultado['imagen']['archivo']

                # Cargar máscaras NPZ
                mascaras_files = resultado['deteccion']['mascaras_guardadas']['archivos']
                mascaras_npz = []

                for mask_info in mascaras_files:
                    npz_path = dir_modelo / "masks" / mask_info['archivo']
                    if npz_path.exists():
                        mask_data = np.load(npz_path)
                        mascaras_npz.append({
                            'npz_data': mask_data,
                            'metadata': mask_info
                        })

                imagenes_data[nombre_imagen] = {
                    'json_data': resultado,
                    'mascaras': mascaras_npz
                }

            resultados[modelo] = {
                'metadatos': json_data.get('metadata', {}),
                'imagenes': imagenes_data
            }

            num_imagenes = len(imagenes_data)
            total_mascaras = sum(len(img['mascaras']) for img in imagenes_data.values())
            print(f"   ✅ {num_imagenes} imágenes, {total_mascaras} máscaras")

        self.datos_cargados = resultados

        print(f"\n{'='*80}")
        print(f"✅ CARGA COMPLETADA")
        print(f"   Modelos cargados: {len(resultados)}")
        print(f"{'='*80}\n")

        return resultados

    def obtener_lista_imagenes(self) -> List[str]:
        """Obtiene lista de nombres de imágenes procesadas"""
        imagenes = set()
        for modelo_data in self.datos_cargados.values():
            imagenes.update(modelo_data['imagenes'].keys())
        return sorted(list(imagenes))

    def obtener_modelos_disponibles(self) -> List[str]:
        """Obtiene lista de modelos cargados"""
        return list(self.datos_cargados.keys())

print("✅ Clase CargadorResultados definida")

✅ Clase CargadorResultados definida


In [4]:
# GENERADOR DE MÁSCARAS INDIVIDUALES
# =============================================================================

class GeneradorMascarasIndividuales:
    """Genera visualizaciones de máscaras individuales"""

    def __init__(self, config: ConfigVisualizador):
        self.config = config

    def _redimensionar_mascara_y_bbox(self, segmentation, bbox, imagen_shape):
        """
        Redimensiona máscara y bbox al tamaño de la imagen original

        SOLUCIÓN AL ERROR: Esta función centraliza el redimensionamiento
        """
        h_orig, w_orig = imagen_shape[:2]
        h_mask, w_mask = segmentation.shape

        # Si las dimensiones coinciden, no hacer nada
        if (h_mask == h_orig) and (w_mask == w_orig):
            return segmentation, bbox

        # Redimensionar máscara
        segmentation_resized = cv2.resize(
            segmentation.astype(np.uint8),
            (w_orig, h_orig),
            interpolation=cv2.INTER_NEAREST
        ).astype(bool)

        # Ajustar bbox proporcionalmente
        scale_x = w_orig / w_mask
        scale_y = h_orig / h_mask
        bbox_ajustado = [
            int(bbox[0] * scale_x),
            int(bbox[1] * scale_y),
            int(bbox[2] * scale_x),
            int(bbox[3] * scale_y)
        ]

        return segmentation_resized, bbox_ajustado

    def generar_para_imagen(self, nombre_imagen: str,
                           datos_modelos: Dict,
                           ruta_imagen_original: Path) -> Dict[str, List[Path]]:
        """Genera todas las máscaras individuales para una imagen"""

        # Crear directorio para esta imagen
        dir_imagen = self.config.OUTPUT_PATH / "por_imagen" / Path(nombre_imagen).stem
        dir_imagen.mkdir(parents=True, exist_ok=True)
        dir_personas = dir_imagen / "personas_individuales"
        dir_personas.mkdir(exist_ok=True)

        # Cargar imagen original
        imagen_original = cv2.imread(str(ruta_imagen_original))
        if imagen_original is None:
            print(f"❌ No se pudo cargar: {ruta_imagen_original}")
            return {}

        imagen_original = cv2.cvtColor(imagen_original, cv2.COLOR_BGR2RGB)

        archivos_generados = {
            'mascaras': [],
            'overlays': [],
            'recortes': [],
            'alphas': []
        }

        # Guardar original
        ruta_original = dir_imagen / "00_original.png"
        Image.fromarray(imagen_original).save(ruta_original)

        # Procesar cada modelo
        for modelo, datos in datos_modelos.items():
            if nombre_imagen not in datos['imagenes']:
                continue

            mascaras = datos['imagenes'][nombre_imagen]['mascaras']

            for idx, mask_data in enumerate(mascaras):
                npz = mask_data['npz_data']
                metadata = mask_data['metadata']

                segmentation = npz['segmentation']
                bbox = npz['bbox']
                confidence = float(npz['person_confidence'])

                base_name = f"{modelo}_person_{idx:02d}"

                # ✅ CORRECCIÓN: Redimensionar una sola vez
                segmentation, bbox = self._redimensionar_mascara_y_bbox(
                    segmentation, bbox, imagen_original.shape
                )

                # 1. Máscara sola
                if self.config.GENERAR_MASCARAS_INDIVIDUALES:
                    mask_img = (segmentation * 255).astype(np.uint8)
                    ruta_mask = dir_personas / f"{base_name}_mask.png"
                    cv2.imwrite(str(ruta_mask), mask_img)
                    archivos_generados['mascaras'].append(ruta_mask)

                # 2. Overlay sobre original
                if self.config.GENERAR_OVERLAYS:
                    overlay = imagen_original.copy()
                    color = self.config.COLORES_MODELOS.get(modelo, (0, 255, 0))
                    overlay[segmentation == 1] = color

                    blended = cv2.addWeighted(
                        imagen_original, 1 - self.config.ALPHA_OVERLAY,
                        overlay, self.config.ALPHA_OVERLAY,
                        0
                    )

                    # Añadir bounding box y texto
                    x, y, w, h = bbox
                    cv2.rectangle(blended, (x, y), (x+w, y+h), color, 2)

                    texto = f"{modelo.upper()} | Conf: {confidence:.2f}"
                    cv2.putText(blended, texto, (x, y-10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    ruta_overlay = dir_personas / f"{base_name}_overlay.png"
                    Image.fromarray(blended).save(ruta_overlay)
                    archivos_generados['overlays'].append(ruta_overlay)

                # 3. Recorte
                if self.config.GENERAR_RECORTES:
                    x, y, w, h = bbox
                    recorte = imagen_original[y:y+h, x:x+w].copy()

                    ruta_recorte = dir_personas / f"{base_name}_cutout.png"
                    Image.fromarray(recorte).save(ruta_recorte)
                    archivos_generados['recortes'].append(ruta_recorte)

                # 4. Recorte con alpha
                if self.config.GENERAR_ALPHA:
                    x, y, w, h = bbox
                    recorte = imagen_original[y:y+h, x:x+w].copy()
                    mask_crop = segmentation[y:y+h, x:x+w]

                    rgba = np.zeros((recorte.shape[0], recorte.shape[1], 4), dtype=np.uint8)
                    rgba[:, :, :3] = recorte
                    rgba[:, :, 3] = mask_crop * 255

                    ruta_alpha = dir_personas / f"{base_name}_alpha.png"
                    Image.fromarray(rgba, mode='RGBA').save(ruta_alpha)
                    archivos_generados['alphas'].append(ruta_alpha)

        return archivos_generados

print("✅ Clase GeneradorMascarasIndividuales definida (CORREGIDA)")

✅ Clase GeneradorMascarasIndividuales definida (CORREGIDA)


In [6]:
# GENERADOR DE COMPARATIVAS
# =============================================================================

class GeneradorComparativas:
    """Genera visualizaciones comparativas entre modelos"""

    def __init__(self, config: ConfigVisualizador):
        self.config = config

    def _redimensionar_mascara_y_bbox(self, segmentation, bbox, imagen_shape):
        """Redimensiona máscara y bbox al tamaño de la imagen original"""
        h_orig, w_orig = imagen_shape[:2]
        h_mask, w_mask = segmentation.shape

        if (h_mask == h_orig) and (w_mask == w_orig):
            return segmentation, bbox

        segmentation_resized = cv2.resize(
            segmentation.astype(np.uint8),
            (w_orig, h_orig),
            interpolation=cv2.INTER_NEAREST
        ).astype(bool)

        scale_x = w_orig / w_mask
        scale_y = h_orig / h_mask
        bbox_ajustado = [
            int(bbox[0] * scale_x),
            int(bbox[1] * scale_y),
            int(bbox[2] * scale_x),
            int(bbox[3] * scale_y)
        ]

        return segmentation_resized, bbox_ajustado

    def comparar_modelos_para_imagen(self, nombre_imagen: str,
                                     datos_modelos: Dict,
                                     ruta_imagen_original: Path) -> Optional[Path]:
        """Genera comparativa de todos los modelos para una imagen"""

        # Cargar imagen original
        imagen_original = cv2.imread(str(ruta_imagen_original))
        if imagen_original is None:
            return None

        imagen_original = cv2.cvtColor(imagen_original, cv2.COLOR_BGR2RGB)

        # Filtrar modelos disponibles
        modelos_disponibles = []
        for modelo, datos in datos_modelos.items():
            if nombre_imagen in datos['imagenes']:
                modelos_disponibles.append(modelo)

        if not modelos_disponibles:
            return None

        # Crear figura
        n_modelos = len(modelos_disponibles)
        n_cols = min(n_modelos + 1, 3)
        n_rows = (n_modelos + 1 + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 6*n_rows))
        if n_rows == 1 and n_cols == 1:
            axes = np.array([[axes]])
        elif n_rows == 1 or n_cols == 1:
            axes = axes.reshape(n_rows, n_cols)

        fig.suptitle(f'Comparativa Modelos SAM 2.0 - {nombre_imagen}',
                    fontsize=16, fontweight='bold')

        axes_flat = axes.flatten()

        # Mostrar original
        axes_flat[0].imshow(imagen_original)
        axes_flat[0].set_title('Original', fontsize=14, fontweight='bold')
        axes_flat[0].axis('off')

        # Mostrar cada modelo
        for idx, modelo in enumerate(modelos_disponibles, start=1):
            ax = axes_flat[idx]

            mascaras = datos_modelos[modelo]['imagenes'][nombre_imagen]['mascaras']

            # Crear overlay
            overlay = imagen_original.copy()
            color = self.config.COLORES_MODELOS.get(modelo, (0, 255, 0))

            # ✅ CORRECCIÓN: Un solo bucle para overlay
            for mask_data in mascaras:
                segmentation = mask_data['npz_data']['segmentation']

                # Redimensionar si es necesario
                segmentation, _ = self._redimensionar_mascara_y_bbox(
                    segmentation, mask_data['npz_data']['bbox'], imagen_original.shape
                )

                overlay[segmentation == 1] = color

            # Blend
            blended = cv2.addWeighted(
                imagen_original, 1 - self.config.ALPHA_OVERLAY,
                overlay, self.config.ALPHA_OVERLAY,
                0
            )

            # Dibujar bounding boxes
            for mask_data in mascaras:
                segmentation = mask_data['npz_data']['segmentation']
                bbox = mask_data['npz_data']['bbox']
                confidence = float(mask_data['npz_data']['person_confidence'])

                # Redimensionar
                _, bbox = self._redimensionar_mascara_y_bbox(
                    segmentation, bbox, imagen_original.shape
                )

                x, y, w, h = bbox
                rect = mpatches.Rectangle(
                    (x, y), w, h,
                    fill=False, edgecolor=np.array(color)/255, linewidth=2
                )
                ax.add_patch(rect)

                ax.text(x, y-5, f'{confidence:.2f}',
                       color=np.array(color)/255, fontsize=10,
                       fontweight='bold',
                       bbox=dict(facecolor='black', alpha=0.7, pad=2))

            ax.imshow(blended)
            ax.set_title(f'{modelo.upper()}\n{len(mascaras)} personas',
                        fontsize=12, fontweight='bold')
            ax.axis('off')

        # Ocultar axes sobrantes
        for idx in range(len(modelos_disponibles) + 1, len(axes_flat)):
            axes_flat[idx].axis('off')

        plt.tight_layout()

        # Guardar
        dir_imagen = self.config.OUTPUT_PATH / "por_imagen" / Path(nombre_imagen).stem
        ruta_salida = dir_imagen / "03_comparativa_modelos.png"
        plt.savefig(ruta_salida, dpi=150, bbox_inches='tight')
        plt.close()

        return ruta_salida

print("✅ Clase GeneradorComparativas definida (CORREGIDA)")

✅ Clase GeneradorComparativas definida (CORREGIDA)


In [7]:
# CARGAR EJECUCIÓN
# =============================================================================

# Buscar ejecuciones disponibles
ejecutiones = sorted([d.name for d in config.RESULTADOS_PATH.glob("*_evaluacion_sam2")])

if not ejecutiones:
    print("❌ No se encontraron ejecuciones")
else:
    print(f"✅ Ejecuciones encontradas: {len(ejecutiones)}")
    for i, ej in enumerate(ejecutiones, 1):
        print(f"   [{i}] {ej}")

    # Seleccionar la más reciente
    directorio_ejecucion = ejecutiones[-1]
    print(f"\n📂 Seleccionada: {directorio_ejecucion}")

    # Cargar datos
    ruta_ejecucion = config.RESULTADOS_PATH / directorio_ejecucion
    cargador = CargadorResultados(ruta_ejecucion)
    datos_modelos = cargador.cargar_ejecucion_completa()

    # Obtener lista de imágenes
    nombres_imagenes = cargador.obtener_lista_imagenes()
    rutas_imagenes = {}

    for nombre in nombres_imagenes:
        ruta_img = config.IMAGENES_PATH / nombre
        if ruta_img.exists():
            rutas_imagenes[nombre] = ruta_img

    print(f"\n📸 Imágenes disponibles: {len(rutas_imagenes)}")
    for nombre in sorted(rutas_imagenes.keys()):
        print(f"   • {nombre}")

✅ Ejecuciones encontradas: 1
   [1] 20251005_192422_evaluacion_sam2

📂 Seleccionada: 20251005_192422_evaluacion_sam2

CARGANDO RESULTADOS DE: 20251005_192422_evaluacion_sam2

📂 Cargando modelo: tiny
   ✅ 6 imágenes, 46 máscaras
📂 Cargando modelo: small
   ✅ 6 imágenes, 78 máscaras
📂 Cargando modelo: base_plus
   ✅ 6 imágenes, 93 máscaras
📂 Cargando modelo: large
   ✅ 6 imágenes, 107 máscaras

✅ CARGA COMPLETADA
   Modelos cargados: 4


📸 Imágenes disponibles: 6
   • 1.jpg
   • 2.jpg
   • 3.jpg
   • 5.jpg
   • 6.jpg
   • 7.jpg


In [8]:
# GENERAR MÁSCARAS INDIVIDUALES
# =============================================================================

gen_mascaras = GeneradorMascarasIndividuales(config)

print(f"\n{'='*80}")
print("GENERANDO MÁSCARAS INDIVIDUALES")
print(f"{'='*80}\n")

resultados_mascaras = {}

for nombre_imagen in tqdm(rutas_imagenes.keys(), desc="Procesando imágenes"):
    ruta_img = rutas_imagenes[nombre_imagen]

    archivos = gen_mascaras.generar_para_imagen(
        nombre_imagen, datos_modelos, ruta_img
    )

    resultados_mascaras[nombre_imagen] = archivos

print(f"\n✅ Máscaras individuales generadas")


GENERANDO MÁSCARAS INDIVIDUALES



Procesando imágenes:   0%|          | 0/6 [00:00<?, ?it/s]


✅ Máscaras individuales generadas


In [9]:
# GENERAR COMPARATIVAS POR IMAGEN
# =============================================================================

gen_comparativas = GeneradorComparativas(config)

print(f"\n{'='*80}")
print("GENERANDO COMPARATIVAS")
print(f"{'='*80}\n")

for nombre_imagen in tqdm(rutas_imagenes.keys(), desc="Comparativas"):
    ruta_img = rutas_imagenes[nombre_imagen]

    ruta_comp = gen_comparativas.comparar_modelos_para_imagen(
        nombre_imagen, datos_modelos, ruta_img
    )

    if ruta_comp:
        print(f"✅ {nombre_imagen} → {ruta_comp.name}")

print(f"\n✅ Comparativas generadas")


GENERANDO COMPARATIVAS



Comparativas:   0%|          | 0/6 [00:00<?, ?it/s]

✅ 1.jpg → 03_comparativa_modelos.png
✅ 2.jpg → 03_comparativa_modelos.png
✅ 3.jpg → 03_comparativa_modelos.png
✅ 5.jpg → 03_comparativa_modelos.png
✅ 6.jpg → 03_comparativa_modelos.png
✅ 7.jpg → 03_comparativa_modelos.png

✅ Comparativas generadas


In [10]:
# VER RESULTADOS
# =============================================================================

print(f"\n{'='*80}")
print("📊 RESUMEN DE RESULTADOS")
print(f"{'='*80}\n")

print(f"📁 Directorio de salida: {config.OUTPUT_PATH}")
print(f"\n🎨 ARCHIVOS GENERADOS:")

# Contar archivos
for nombre_imagen, archivos in resultados_mascaras.items():
    print(f"\n📸 {nombre_imagen}:")
    for tipo, lista in archivos.items():
        if lista:
            print(f"   • {tipo}: {len(lista)} archivos")

print(f"\n{'='*80}")
print("✅ PROCESO COMPLETADO")
print(f"{'='*80}\n")


📊 RESUMEN DE RESULTADOS

📁 Directorio de salida: /content/drive/MyDrive/TFM/sam2/visualizaciones_avanzadas

🎨 ARCHIVOS GENERADOS:

📸 1.jpg:
   • mascaras: 23 archivos
   • overlays: 23 archivos
   • recortes: 23 archivos
   • alphas: 23 archivos

📸 2.jpg:
   • mascaras: 77 archivos
   • overlays: 77 archivos
   • recortes: 77 archivos
   • alphas: 77 archivos

📸 3.jpg:
   • mascaras: 36 archivos
   • overlays: 36 archivos
   • recortes: 36 archivos
   • alphas: 36 archivos

📸 5.jpg:
   • mascaras: 43 archivos
   • overlays: 43 archivos
   • recortes: 43 archivos
   • alphas: 43 archivos

📸 6.jpg:
   • mascaras: 96 archivos
   • overlays: 96 archivos
   • recortes: 96 archivos
   • alphas: 96 archivos

📸 7.jpg:
   • mascaras: 49 archivos
   • overlays: 49 archivos
   • recortes: 49 archivos
   • alphas: 49 archivos

✅ PROCESO COMPLETADO

